Tema2
Lidia Istrate
342C4


Functions for reading the input:

In [104]:
import re


#Representation
CONSTANT = "CONSTANT"
VARIABLE = "VARIABLE"
FUNCTION = "FUNCTION"
ATOM = "ATOM"
CONDITION = "CONDITION"
RULE = "RULE"
QUESTION = "QUESTION"
ANSWER = "ANSWER"
COMMENT = "COMMENT"




def make_const(value):
    return (CONSTANT, value)


def make_var(name):
    return (VARIABLE, name)


def make_function_call(function, args):
    return (FUNCTION, function, args)


def make_atom(predicate, args, atom_string):
    return (ATOM, predicate, args, atom_string)


def make_question(predicate, args, q_string):
    return (QUESTION, predicate, args, q_string)

#conditie
def make_condition(conditions):
    return (CONDITION, CONDITION, conditions)

def make_rule(atom, conditions, rule_string):
    return (RULE, atom, conditions, rule_string)


def is_constant(f):
    return f[0] == CONSTANT


def is_variable(f):
    return f[0] == VARIABLE


def is_function_call(f):
    return f[0] == FUNCTION


def is_atom(f):
    return f[0] == ATOM


def is_question(f):
    return f[0] == QUESTION


def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

def is_sentence(f):
    return is_atom(f) or f[0] == CONDITION


def has_args(f):
    return is_function_call(f) or is_sentence(f)


def get_value(f):
    if is_constant(f):
        return f[1]
    return None


def get_name(f):
    if is_variable(f):
        return f[1]
    return None


def get_head(f):
    if is_function_call(f) or is_atom(f):
        return f[1]
    if f[0] == CONDITION:
        return 'A'
    return None


def get_args(f):
    if has_args(f):
        return f[2]
    return []

def replace_args(formula, new_args):
    return (formula[0], formula[1], new_args)

def get_type(line):
    
    if line[0] == "?":
        return QUESTION
    elif ":" in line and line[0] != ":":
        return RULE
    elif line[0] == ":":
        return ANSWER
    elif line[0] == "#":
        return COMMENT
    else:
        return ATOM
        

facts = []
rules = []
questions = []

def process_atom(line):
        
    predicate = ''
    current_string = ''
    args = []
    index = 0
    i = 0
    while i < len(line):
        
        if line[i] == '(' and predicate == '':
            predicate = current_string

            current_string = ''
            index = i+1
        
        elif line[i] == ',' or line[i] == ')':
            if current_string != '':
                if '?' in current_string:
                    var = make_var(current_string)
                    args.append(var)

                else:
                    const = make_const(current_string)
                    args.append(const)

                
            current_string = ''
        
        elif line[i] == ' ':
            index = i+1
        elif line[i] == '(' and predicate != '':
            j = index
            function_string = ''
            
            count = 0
            first_paranth = False
            ok = False
            
            while ok == False:
                if line[j] == ')':
                    count -= 1
                    if first_paranth == True and count == 0:
                        ok = True
                if line[j] == '(':
                    count += 1
                    first_paranth = True

                function_string += line[j]
                j = j + 1
            

            atom_f = process_atom(function_string)
            function = make_function_call(atom_f[1], atom_f[2])
            args.append(function)
            current_string = ''
            i = j-1

                 
        else:
            current_string = current_string + line[i]
        i = i + 1
            

    return make_atom(predicate, args, line.strip(' \n'))
    
    
def process_rule(line):
    conditions = []
    i = 0
    
    current_string = ''
    while (line[i] + line[i+1] != ' :'):
        current_string = current_string + line[i]
        i = i + 1
    atom = process_atom(current_string)
    i = i + 3

    
    while i < len(line):
        current_string = ''
        count = 0
        first_paranth = False
        ok = False

        while ok == False:
            if line[i] == ')':
                count -= 1
                if first_paranth == True and count == 0:
                    ok = True
            if line[i] == '(':
                count += 1
                first_paranth = True

            current_string += line[i]
            i = i + 1    

        condition = process_atom(current_string)
        conditions.append(condition)

        
        i = i + 2
    
    return make_rule(atom, make_condition(conditions), line.strip(' \n'))
    
    
def read_input(filename):
    
    f = open(filename, "r")
    for line in f:
        if line and line != '\n':
            full_line = line
            line = line.split()
            

            line_type = get_type(line)
            
            if line_type == ATOM:
                atom = process_atom(full_line)
                facts.append(atom)
            elif line_type == QUESTION:
                q_line = full_line[2:len(full_line)]
                atom = process_atom(q_line)
                question = make_question(atom[1], atom[2], "? " + atom[3])
                questions.append(question)
            elif line_type == RULE:
                rule = process_rule(full_line)
                rules.append(rule)            
                
    f.close()

            
            
            


def main():
    read_input("teste/test1.txt")
    for fact in facts:
        print(fact)
        
    for rule in rules:
        print(rule)
        
    for question in questions:
        print(question)        

main()

('ATOM', 'P', [('CONSTANT', '1')], 'P(1)')
('ATOM', 'P', [('VARIABLE', '?2'), ('FUNCTION', 'U', [('CONSTANT', '3'), ('FUNCTION', 'V', [('CONSTANT', '4'), ('CONSTANT', '5')]), ('CONSTANT', '6')]), ('CONSTANT', '7')], 'P(?2, U(3, V(4, 5), 6), 7)')
('RULE', ('ATOM', 'r', [('VARIABLE', '?a'), ('CONSTANT', '8')], 'r(?a, 8)'), ('CONDITION', 'CONDITION', [('ATOM', 'P', [('VARIABLE', '?a')], 'P(?a)'), ('ATOM', 'Q', [('VARIABLE', '?b'), ('CONSTANT', 'hello')], 'Q(?b, hello)'), ('ATOM', 'P', [('VARIABLE', '?2'), ('FUNCTION', 'U', [('CONSTANT', '3'), ('FUNCTION', 'V', [('CONSTANT', '4'), ('CONSTANT', '5')]), ('CONSTANT', '6')]), ('CONSTANT', '7')], 'P(?2, U(3, V(4, 5), 6), 7)'), ('ATOM', 'T', [('VARIABLE', '?i')], 'T(?i)')]), 'r(?a, 8) : P(?a), Q(?b, hello), P(?2, U(3, V(4, 5), 6), 7), T(?i)')
